In [ ]:
# Modified using code from Matthew Whitley

import requests
from bs4 import BeautifulSoup
import os
import re
import pandas as pd
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
from tqdm import tqdm
import glob
import datetime as dt
import geopandas as gpd
import zipfile

In [ ]:
def get_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a')
    return links

def get_all_files(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a')
    out = []
    for link in links[5:]:
        href = link.get('href')
        subdir_url = url + href
        # print(subdir_url)
        if subdir_url.endswith('/'):  # Check if the link is a directory
            out.append(get_all_files(subdir_url))
        else: # if not add download link to outlist
            # print('appended')
            out.append(subdir_url)  
    return out

def flatten_list(lst):
    flattened = []
    for item in lst:
        if isinstance(item, list):
            flattened.extend(flatten_list(item))
        else:
            flattened.append(item)
    return flattened

In [ ]:
url = 'https://ftp.wildfire.gov/public/incident_specific_data/calif_s/'
region = 'calif_s'
fyeardirs = ['2020_Incidents/']

In [ ]:
# Find all the links on the page
allout = []
links = get_links(url)
for link in links[5:]:
    href = link.get('href')
    print(href)
    if href not in fyeardirs:
        continue
    subdir_url = url + href
    sublinks = get_links(subdir_url)
    for sl in sublinks[5:]:
        href = sl.get('href')
        su = subdir_url+href
        # force IR dir
        irdir = su+'IR/' # THIS IS NOT CONSISTENT AMONG REGIONS AND ACROSS YEARS (so please check it) 
        # print(irdir)
        allout.append(get_all_files(irdir))
allout = flatten_list(allout)
print(len(allout))


In [ ]:
import os
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# List of URLs to download
urls = allout

# Directory to save the downloaded files
local_directory = 'C:\\Users\\Desktop\\NIROPS_Data'

# Function to download a file from a URL
def download_file(url):
    local_filename = url.split('/')[-1]  # Extract filename from URL
    path = os.path.join(local_directory, local_filename)
    response = requests.get(url, stream=True)
    total_size_in_bytes = int(response.headers.get('content-length', 0))
    block_size = 1024 # 1 Kibibyte
    with open(path, 'wb') as file, tqdm(
        total=total_size_in_bytes, unit='iB', unit_scale=True, desc=local_filename) as progress_bar:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)

# Using ThreadPoolExecutor to download files concurrently
def download_files(urls):
    with ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(download_file, urls)

# Download all files concurrently
download_files(urls)
